Input: arquivo com a mensagem em claro + Chave RSA pública do destinatário + Algoritmo simétrico

In [13]:
from Crypto.PublicKey import RSA
from Crypto.Cipher import AES
from Crypto.Cipher import DES
from Crypto.Cipher import ARC4
from Crypto.Random import get_random_bytes

In [14]:
class Nigma:
    def __init__(self):
        pass

    def generateRandomKey(self, sizeKey):
        return get_random_bytes(sizeKey)

    def encryptAES(self, message, key, initializationVector, modeOperation):
        if modeOperation == AES.MODE_CBC:
            cipher = AES.new(key, modeOperation, initializationVector)
            sizeBlock = AES.block_size
            messagePadded = message + (sizeBlock - len(message) % sizeBlock) * b"\x00"
            return cipher.encrypt(messagePadded)
        elif modeOperation == AES.MODE_ECB:
            cipher = AES.new(key, modeOperation)

    
    def encryptDES(self, message, key, modeOperation):
        if modeOperation == 0:
            pass
        elif modeOperation == 1:
            pass
        elif modeOperation == 2:
            pass

    def encryptRC4(self, message, key, modeOperation):
        if modeOperation == 0:
            pass
        elif modeOperation == 1:
            pass
        elif modeOperation == 2:
            pass

    def decryptAES(self,cryptogram, key, tag, modeOperation):
        cipher = AES.new(key, modeOperation, dados[0])
        return cipher.decrypt_and_verify(cryptogram, tag)
    
    def decryptDES(self,cryptogram, key, tag, modeOperation):
        pass
    
    def decryptRC4(self,cryptogram, key, tag, modeOperation):
        pass

    def __str__(self):
        return "Hi, I'm Nigma!"

In [15]:
def generatePairKeys(sizeKey=1024):
    key = RSA.generate(sizeKey)
    privKey = key.export_key()
    publiKey = key.publickey().export_key()
    return (privKey, publiKey)

In [16]:
def main():
    maquina = Nigma()
    message = b"Hello World, GAROTOOOOOOOO"
    randomKey = maquina.generateRandomKey(16)
    initializationVector = maquina.generateRandomKey(16)
    cryptogram = maquina.encryptAES(message, randomKey, initializationVector, AES.MODE_CBC)
    print(cryptogram)


if __name__ == "__main__":
    main()

b'\x18#\xaey\x1e\xce\x0e\xd8\xca\xcb\x12\xd4\t%HW\xb3\x8c\xe4W\xfb7 \xb3\x88&\xa05\x84\x06X='
